In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE68600"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE68600"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE68600.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE68600.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE68600.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_cho-00156: Gene Expression in Ovarian Cancer Reflects Both Morphology and Biological Behavior"
!Series_summary	"Biologically and clinically meaningful tumor classification schemes have long been sought. Some malignant epithelial neoplasms, such as those in the thyroid and endometrium, exhibit more than one pattern of differentiation, each associated with distinctive clinical features and treatments. In other tissues, all carcinomas, regardless of morphological type, are treated as though they represent a single disease. To better understand the biological and clinical features seen in the four major histological types of ovarian carcinoma (OvCa), we analyzed gene expression in 113 ovarian epithelial tumors using oligonucleotide microarrays. Global views of the variation in gene expression were obtained using PCA. These analyses show that mucinous and clear cell OvCas can be readily distinguished from serous OvCas based on their gene expres

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from oligonucleotide microarrays
# Specifically, it mentions "Gene Expression in Ovarian Cancer" and "gene expression profiles"
is_gene_available = True

# 2.1 Data Availability
# For trait (Endometrioid Cancer), we can determine this from the histology information in row 4
trait_row = 4  # corresponds to histology data

# Age data is not available in the sample characteristics
age_row = None

# Gender data appears to be available in row 0, but it shows only "Sex: F" which means all samples are female
# Since this is a constant feature (all samples are female), we consider it not available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert histology value to binary indicator for Endometrioid Cancer."""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Check if endometrioid is in the histology
    if 'endometrioid' in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to numerical format."""
    # Function defined but not used as age data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)."""
    # Function defined but not used as gender data is constant (all female)
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value in ['f', 'female']:
        return 0
    elif value in ['m', 'male']:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering based on gene and trait availability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("Preview of processed clinical data:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of processed clinical data:
{'GSM1676602': [0.0], 'GSM1676603': [0.0], 'GSM1676604': [1.0], 'GSM1676605': [1.0], 'GSM1676606': [1.0], 'GSM1676607': [1.0], 'GSM1676608': [1.0], 'GSM1676609': [1.0], 'GSM1676610': [1.0], 'GSM1676611': [1.0], 'GSM1676612': [1.0], 'GSM1676613': [1.0], 'GSM1676614': [1.0], 'GSM1676615': [0.0], 'GSM1676616': [0.0], 'GSM1676617': [0.0], 'GSM1676618': [0.0], 'GSM1676619': [0.0], 'GSM1676620': [0.0], 'GSM1676621': [0.0], 'GSM1676622': [0.0], 'GSM1676623': [0.0], 'GSM1676624': [0.0], 'GSM1676625': [0.0], 'GSM1676626': [0.0], 'GSM1676627': [0.0], 'GSM1676628': [0.0], 'GSM1676629': [0.0], 'GSM1676630': [0.0], 'GSM1676631': [0.0], 'GSM1676632': [0.0], 'GSM1676633': [0.0], 'GSM1676634': [0.0], 'GSM1676635': [0.0], 'GSM1676636': [0.0], 'GSM1676637': [0.0], 'GSM1676638': [0.0], 'GSM1676639': [0.0], 'GSM1676640': [0.0], 'GSM1676641': [0.0], 'GSM1676642': [0.0], 'GSM1676643': [0.0], 'GSM1676644': [0.0], 'GSM1676645': [0.0], 'GSM1676646': [1.0], 'GSM1676647': [1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 71
Header line: "ID_REF"	"GSM1676602"	"GSM1676603"	"GSM1676604"	"GSM1676605"	"GSM1676606"	"GSM1676607"	"GSM1676608"	"GSM1676609"	"GSM1676610"	"GSM1676611"	"GSM1676612"	"GSM1676613"	"GSM1676614"	"GSM1676615"	"GSM1676616"	"GSM1676617"	"GSM1676618"	"GSM1676619"	"GSM1676620"	"GSM1676621"	"GSM1676622"	"GSM1676623"	"GSM1676624"	"GSM1676625"	"GSM1676626"	"GSM1676627"	"GSM1676628"	"GSM1676629"	"GSM1676630"	"GSM1676631"	"GSM1676632"	"GSM1676633"	"GSM1676634"	"GSM1676635"	"GSM1676636"	"GSM1676637"	"GSM1676638"	"GSM1676639"	"GSM1676640"	"GSM1676641"	"GSM1676642"	"GSM1676643"	"GSM1676644"	"GSM1676645"	"GSM1676646"	"GSM1676647"	"GSM1676648"	"GSM1676649"	"GSM1676650"	"GSM1676651"	"GSM1676652"	"GSM1676653"	"GSM1676654"	"GSM1676655"	"GSM1676656"	"GSM1676657"	"GSM1676658"	"GSM1676659"	"GSM1676660"	"GSM1676661"	"GSM1676662"	"GSM1676663"	"GSM1676664"	"GSM1676665"	"GSM1676666"	"GSM1676667"	"GSM1676668"	"GSM1676669"	"GSM1676670"	"GSM1676671"	"GSM1676672"	"GSM1676673"	"GSM1676674"	

### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data, we can see identifiers like "A28102_at", "AB000114_at", etc.
# These appear to be probe IDs from an Affymetrix microarray platform, not standard human gene symbols.
# Affymetrix probe IDs typically need to be mapped to human gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE68600
Line 6: !Series_title = caArray_cho-00156: Gene Expression in Ovarian Cancer Reflects Both Morphology and Biological Behavior
Line 7: !Series_geo_accession = GSE68600
Line 8: !Series_status = Public on May 07 2015
Line 9: !Series_submission_date = May 06 2015
Line 10: !Series_last_update_date = Jul 08 2016
Line 11: !Series_pubmed_id = 12183431
Line 12: !Series_summary = Biologically and clinically meaningful tumor classification schemes have long been sought. Some malignant epithelial neoplasms, such as those in the thyroid and endometrium, exhibit more than one pattern of differentiation, each associated with distinctive clinical features and treatments. In other tissues, all carcinomas, regardles


Gene annotation preview:
{'ID': ['A28102_at', 'AB000114_at', 'AB000115_at', 'AB000220_at', 'AB000381_s_at'], 'GB_ACC': ['A28102', 'AB000114', 'AB000115', 'AB000220', 'AB000381'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'Target Description': ['A28102, class A, 20 probes, 16 in A28102cds 986-1442: 4 in reverseSequence, 1546-1582, Human GABAa receptor alpha-3 subunit.', 'AB000114, class A, 20 probes, 20 in AB000114 1818-2208, Human mRNA for osteomodulin, complete cds', 'AB000115, class A, 20 probes, 20 in AB000115 1469-1973, Human mRNA, complete cds', 'AB000220, class A, 20 probes, 20 in AB00

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation contain the gene identifier and gene symbol
# Based on the preview, we can see:
# - 'ID' column contains identifiers like 'A28102_at' which match the gene expression data
# - 'Gene Symbol' column contains standard gene symbols like 'GABRA3'

# 2. Extract the gene mapping dataframe from the gene annotation
gene_mapping = gene_annotation[['ID', 'Gene Symbol']].rename(columns={'Gene Symbol': 'Gene'})

# Show the first few mappings to verify
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# The apply_gene_mapping function handles the division of probe values among multiple genes
# and summing up all probe values for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the result
print("\nGene expression data preview (first 5 genes, 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the processed gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
              ID    Gene
0      A28102_at  GABRA3
1    AB000114_at     OMD
2    AB000115_at  IFI44L
3    AB000220_at  SEMA3C
4  AB000381_s_at     GML



Gene expression data preview (first 5 genes, 5 samples):
       GSM1676602  GSM1676603  GSM1676604  GSM1676605  GSM1676606
Gene                                                             
A2M        1811.0      3908.0      3826.0      4674.0      1638.0
AADAC        34.0        81.0       225.0        16.0       155.0
AAMP       2176.0      1885.0      3466.0      1332.0      2475.0
AANAT       998.0       532.0       844.0       556.0      1340.0
AARS       1995.0      3358.0      1931.0      2293.0      2812.0


Gene expression data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE68600.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (5921, 113)
First few genes with their expression values after normalization:
       GSM1676602  GSM1676603  GSM1676604  GSM1676605  GSM1676606  GSM1676607  \
Gene                                                                            
A2M        1811.0      3908.0      3826.0      4674.0      1638.0      4061.0   
AADAC        34.0        81.0       225.0        16.0       155.0        63.0   
AAMP       2176.0      1885.0      3466.0      1332.0      2475.0      2772.0   
AANAT       998.0       532.0       844.0       556.0      1340.0      1086.0   
AARS1      1995.0      3358.0      1931.0      2293.0      2812.0      4343.0   

       GSM1676608  GSM1676609  GSM1676610  GSM1676611  ...  GSM1676705  \
Gene                                                   ...               
A2M        1715.0     14602.0      1861.0      2095.0  ...      1433.0   
AADAC        38.0        61.0        64.0        76.0  ...       126.0   
AAMP       1978.0      1606.0 

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE68600.csv
Raw clinical data shape: (7, 114)
Clinical features:
                     GSM1676602  GSM1676603  GSM1676604  GSM1676605  \
Endometrioid_Cancer         0.0         0.0         1.0         1.0   

                     GSM1676606  GSM1676607  GSM1676608  GSM1676609  \
Endometrioid_Cancer         1.0         1.0         1.0         1.0   

                     GSM1676610  GSM1676611  ...  GSM1676705  GSM1676706  \
Endometrioid_Cancer         1.0         1.0  ...         1.0         0.0   

                     GSM1676707  GSM1676708  GSM1676709  GSM1676710  \
Endometrioid_Cancer         0.0         0.0         1.0         0.0   

                     GSM1676711  GSM1676712  GSM1676713  GSM1676714  
Endometrioid_Cancer         0.0         0.0         0.0         0.0  

[1 rows x 113 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE68600.csv
Linked

Data shape after handling missing values: (113, 5922)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 38 occurrences. This represents 33.63% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometrioid_Cancer/GSE68600.csv
